<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Video-preprocessing" data-toc-modified-id="Video-preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Video preprocessing</a></span><ul class="toc-item"><li><span><a href="#video-to-frame-conversion" data-toc-modified-id="video-to-frame-conversion-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>video to frame conversion</a></span></li><li><span><a href="#detection-of-face-coordinates" data-toc-modified-id="detection-of-face-coordinates-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>detection of face coordinates</a></span></li><li><span><a href="#face-detection-in-the-frames-and-cropping" data-toc-modified-id="face-detection-in-the-frames-and-cropping-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>face detection in the frames and cropping</a></span></li></ul></li></ul></div>

# Video preprocessing

## video to frame conversion 

In [11]:
# fps is numebr of frames per second
import os
file_ = '../../2021-10-25_10-31-58_c27'
frame_ = 'f'
# command = f'ffmpeg -i ../../2021-10-25_10-31-58_c27.mp4 -vf fps=1/60 ../../frames/f-%03d.jpg'
command = f'ffmpeg -i {file_}.mp4 -vf fps=1/120 ../../frames_2/{frame_}-%03d.jpg'
os.system(command)

ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 12.0.1
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955491330/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955491330/_build_env/bin/pkg-config --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955491330/_build_env/bin/x86_64-apple-darwin13.4.0-c

0

## detection of face coordinates

In [ ]:
import cv2
from pylab import *

frame_path = '../../frames/'
file = 'f-003.jpg'
img = cv2.imread('%s%s' % (frame_path, file))
x = img.shape[1]
y = img.shape[0]
print("x=", x)
print("y=", y)
imshow(img)

This function open a image and you can choose the approximate location of the center of the face, (try to do it within 5 second)

In [12]:
# importing the module
import cv2 
# function to display the coordinates of
# of the points clicked on the image
def click_event(event, x, y, flags, params):
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
        
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', img)
    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
        
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', img)


In [ ]:
frame_path = '../../frames/'
file = 'f-003.jpg'
img = cv2.imread('%s%s' % (frame_path, file))
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)
cv2.waitKey(5000)
cv2.destroyAllWindows()
# crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
# plt.imshow(crop_img)
# plt.show()

By running this over all the videos, we can extract the x and y position of the faces. But, I think all of them probably are in the same spot. So, we might not even need to repeat this step for all the videos. I have run it once and for the video I had, x= 1094 and y = 199. 

## face detection in the frames and cropping

In [13]:
# coordiante of the face (approximately center)
x = 1094
y = 199

In [14]:
from mtcnn import MTCNN # via conda-forge mtccn, via pip mtcnn_cv2
import cv2 # python_opencv
import pandas as pd
import os

def bounding_box_check(faces, x, y):
    # check the center
    for face in faces:
        bounding_box = face['box']
        if (bounding_box[1] < 0):
            bounding_box[1] = 0
        if (bounding_box[0] < 0):
            bounding_box[0] = 0
        if (bounding_box[0] - 50 > x
                or bounding_box[0] + bounding_box[2] + 50 < x):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        if (bounding_box[1] - 50 > y
                or bounding_box[1] + bounding_box[3] + 50 < y):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        return bounding_box

def face_detect(file, detector, frame_path, x_pos, y_pos):
    name = file.replace('.jpg', '').split('-')
    x = x_pos
    y = y_pos
    img = cv2.imread('%s%s' % (frame_path, file ))
    faces = detector.detect_faces(img)
    # check if detected faces
    if (len(faces) == 0):
        print('no face detect: ' + file)
        return  #no face
    bounding_box = bounding_box_check(faces, x, y)
    if (bounding_box == None):
        print('face is not related to given coord: ' + file)
        return
    print(file, " ", bounding_box)
    print(file, " ", x, y)
    crop_img = img[bounding_box[1]:bounding_box[1] + bounding_box[3],
                   bounding_box[0]:bounding_box[0] + bounding_box[2]]
    crop_img = cv2.resize(crop_img, (160, 160))
    cv2.imwrite('%s/frame_' % output_dir + name[0] + '_' + name[1] + '.jpg',
                crop_img)
    

In [ ]:
detect_range = (1, 11)
detector = MTCNN()
output_dir = '../../face_input'  # output directroy to save the files
frame_path = '../../frames/'
for i in range(detect_range[0], detect_range[1]):
    file_name = "f-%03d.jpg" % (i)
    print(file_name)
    if (not os.path.exists('%s%s' % (frame_path, file_name))):
        print('cannot find input: ' + '%s%s' % (frame_path, file_name))
        continue
    face_detect(file_name, detector, frame_path, x, y)